<a href="https://colab.research.google.com/github/kav0001/Capstone-Project-Predicting-Career-Domain-and-Seniority-from-LinkedIn-Profiles/blob/main/PDS_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Rule-based Seniority Classification

The process followed an iterative refinement strategy based on errors. Starting with a simple baseline based on keywords, we gradually incorporated additional linguistic patterns, contextual exceptions, and priority rules for English, German, and partially French positions.

Each iteration was evaluated using accuracy, reproducibility, and F1 score for each class, and targeted error analysis determined subsequent improvements. This approach allowed us to systematically eliminate common confusions between seniority levels (e.g., “senior” and “lead,” ‘management’ and “senior”) while maintaining model interpretability.

After the second iteration of refinement, the model achieved 93% accuracy with balanced performance across all seniority classes. Further refinements involved significantly more complex and fragile rule sets, leading us to a decrease in overall effectiveness and generalizability. Accuracy dropped to 0.53. Given that the model relies solely on job titles without additional contextual information, this level of performance represents a practical limit for a rule-based approach.

Therefore, we choose this version as the final rule-based model and do not make any further refinements.

In [ ]:
import re, unicodedata

def normalize_title(title: str) -> str:
    s = str(title).lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = re.sub(r"[^\w\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

DIRECTOR_PAT = re.compile(r"\b(managing director|director|direktor|directeur)\b")

CLEVEL_PAT = re.compile(r"\b(ceo|cto|cfo|coo|cio|cdo|chief|svp|evp|vp|vice president|president)\b")
ASSIST_PAT = re.compile(r"\b(assistant|executive assistant|assistent|assistentin|assistante)\b")

# Management без "partner" (слишком шумит)
MANAGEMENT_PAT = re.compile(
    r"\b(ceo|cto|cfo|coo|cio|cdo|chief|svp|evp|vp|vice president|president|"
    r"owner|founder|co founder|gruender|grunder|"
    r"geschaftsfuhrer|geschaftsfuhrerin|geschaftsfuhrung|vorstand|vorsitzender)\b"
)
MANAGING_PARTNER_PAT = re.compile(r"\bmanaging partner\b")
PARTNER_CONTEXT_NONMGMT_PAT = re.compile(
    r"\b(business partner|hr business partner|partner manager|partner management|channel partner|partner sales|sales partner)\b"
)

HEAD_PAT = re.compile(r"\bhead\b")
LEAD_PAT = re.compile(r"\b(\w*leiter\w*|\w*leitung\w*|teamlead|team lead|principal|staff)\b")
LEAD_WORD_PAT = re.compile(r"\blead\b")

TEAM_LEAD_EN_PAT = re.compile(r"\bteam lead\b")
PROJECT_LEAD_EN_PAT = re.compile(r"\bproject lead\b")
LEAD_FUNCTIONAL_FOLLOW_PAT = re.compile(r"\blead\s+(engineer|analyst|consultant|specialist|architect|developer|scientist|designer|product owner)\b")

JUNIOR_EXPLICIT_PAT = re.compile(r"\b(junior|jr|praktikant|werkstudent|trainee|intern|internship|apprentice|entry|graduate)\b")
JUNIOR_ROLE_PAT = re.compile(r"\b(mitarbeiterin|mitarbeiter|referentin|referent|assistentin|assistent|analyst)\b")
SENIOR_CUES_PAT = re.compile(r"\b(senior|sr|manager|engineer|consultant|specialist|expert)\b")

def predict_seniority(position: str) -> str:
    s = normalize_title(position)

    if DIRECTOR_PAT.search(s):
        return "Director"

    # ассистенты при C-level обычно НЕ Management в твоей разметке
    if ASSIST_PAT.search(s) and CLEVEL_PAT.search(s):
        return "Senior"

    if (MANAGEMENT_PAT.search(s) or MANAGING_PARTNER_PAT.search(s)):
        if PARTNER_CONTEXT_NONMGMT_PAT.search(s):
            return "Senior"
        return "Management"

    # в твоей разметке "team lead"/"project lead" чаще Senior
    if TEAM_LEAD_EN_PAT.search(s) or PROJECT_LEAD_EN_PAT.search(s):
        return "Senior"
    if LEAD_FUNCTIONAL_FOLLOW_PAT.search(s):
        return "Senior"

    if HEAD_PAT.search(s):
        if ("assistant" in s) or re.search(r"\bmanager\b", s):
            pass
        else:
            return "Lead"

    if LEAD_PAT.search(s) or LEAD_WORD_PAT.search(s):
        return "Lead"

    if JUNIOR_EXPLICIT_PAT.search(s):
        return "Junior"
    if JUNIOR_ROLE_PAT.search(s):
        if re.search(r"\b(senior|sr)\b", s):
            return "Senior"
        return "Junior"

    if SENIOR_CUES_PAT.search(s):
        return "Senior"

    return "Senior"


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


url = "https://drive.google.com/uc?export=download&id=17rxzgzyZbgtBV0gj9Ai1sBmnrrggShZj"
df = pd.read_csv(url)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label"])

test_pred = test_df["text"].map(predict_seniority)
print(classification_report(test_df["label"], test_pred))

              precision    recall  f1-score   support

    Director       0.98      0.96      0.97       197
      Junior       0.91      0.91      0.91        82
        Lead       0.92      0.96      0.94       709
  Management       0.88      0.91      0.90       151
      Senior       0.94      0.89      0.92       747

    accuracy                           0.93      1886
   macro avg       0.93      0.93      0.93      1886
weighted avg       0.93      0.93      0.93      1886



## Rule-based Department Classification

In [36]:
import re
import unicodedata
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
# -----------------------
# 1) Load data
# -----------------------
url_dep = "https://drive.google.com/uc?export=download&id=1oCm72k266iDlpGgwgHua2pQwAw2bKYdW"
df = pd.read_csv(url_dep)
df = df.dropna(subset=["text", "label"]).copy()

# -----------------------
# 2) Normalization
# -----------------------
def normalize_text(s: str) -> str:
    s = str(s).lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))  # remove accents
    s = re.sub(r"[^\w\s]", " ", s)  # punctuation -> space
    s = re.sub(r"\s+", " ", s).strip()
    return s

# -----------------------
# 3) Regex patterns (multilingual-ish)
# Labels in your dataset (11):
# Marketing, Project Management, Administrative, Business Development, Consulting,
# Human Resources, Information Technology, Other, Purchasing, Sales, Customer Support
# -----------------------

IT_PAT = re.compile(
    r"\b(it|information technology|informatik|informatique|software|developer|entwickler|"
    r"engineer|ingenieur|data|ml|ai|devops|cloud|sysadmin|system|security|cyber|"
    r"backend|frontend|fullstack|qa|test(ing)?|sre|database|dba|sap)\b"
)

HR_PAT = re.compile(
    r"\b(hr|human resources|people(\s+ops)?|talent|recruit(ing|er|ment)?|"
    r"personal(|wesen)|personnel|rh\b|ressources humaines|"
    r"bewerbung|hire|hiring|payroll|compensation|benefits)\b"
)

PURCH_PAT = re.compile(
    r"\b(purchas(ing|er)?|procurement|sourcing|einkauf|"
    r"beschaffung|achats|acheteur|approvisionnement)\b"
)

CS_PAT = re.compile(
    r"\b(customer support|support|customer service|service desk|helpdesk|"
    r"kund(en)?support|kundenservice|customer care|success|"
    r"technical support|hotline|call center|after sales|"
    r"beschwerde|rekla(mation)?|ticket)\b"
)

SALES_PAT = re.compile(
    r"\b(sales|vertrieb|vente(s)?|account manager|key account|kamu|"
    r"seller|inside sales|field sales|salesforce|"
    r"commercial\b|commerciaux|business sales)\b"
)

BD_PAT = re.compile(
    r"\b(business development|biz dev|bdm\b|partnership(s)?|alliances?|"
    r"strategic partnerships?|growth\b|expansion|new business|"
    r"partenariat(s)?|entwicklungsgeschaft|geschaftsentwicklung)\b"
)

CONSULTING_PAT = re.compile(
    r"\b(consultant|consulting|beratung|berat(er|ung)?|conseil|"
    r"strategy consultant|management consultant|"
    r"projektberatung|strategie(beratung)?)\b"
)

PM_PAT = re.compile(
    r"\b(project manager|project management|pm\b|pmo\b|programme manager|program manager|"
    r"chef de projet|gestion de projet|projet\b|projekt\b|projektmanager|"
    r"scrum master|product owner|delivery manager|"
    r"project lead|projektleitung|projektleiter)\b"
)

MKT_PAT = re.compile(
    r"\b(marketing|communication|kommunikation|comms\b|branding|brand|"
    r"seo|sea|sem|content|social media|smm\b|pr\b|presse|events?|evenement|"
    r"webmarketing|growth marketing|performance marketing|"
    r"campaign|kampagne|crm\b|newsletter|copywriter|redaktion)\b"
)

ADMIN_PAT = re.compile(
    r"\b(administrative|administration|assistant|assistenz|assistent(in)?|"
    r"office|office management|back office|secretary|sekretar(in)?|"
    r"executive assistant|assistant de direction|assistante de direction|"
    r"operations assistant|buchhaltung|accounting assistant)\b"
)

# -----------------------
# 4) Rule-based predictor
# Important: ordering matters (resolve ambiguity)
# -----------------------
def predict_department(title: str) -> str:
    s = normalize_text(title)

    # 1) IT first (often distinctive)
    if IT_PAT.search(s):
        return "Information Technology"

    # 2) HR
    if HR_PAT.search(s):
        return "Human Resources"

    # 3) Purchasing
    if PURCH_PAT.search(s):
        return "Purchasing"

    # 4) Customer Support (support words can appear elsewhere, but still useful)
    if CS_PAT.search(s):
        return "Customer Support"
        # 5) Sales vs Business Development
    # If both match, prefer BD when partnership/growth words appear
    sales_hit = bool(SALES_PAT.search(s))
    bd_hit = bool(BD_PAT.search(s))
    if bd_hit and not sales_hit:
        return "Business Development"
    if sales_hit and not bd_hit:
        return "Sales"
    if bd_hit and sales_hit:
        return "Business Development"

    # 6) Project Management vs Marketing
    # Many titles contain "project/projet/projekt" but are actually marketing (e.g., "chef de projet marketing")
    # If marketing keywords are present, prefer Marketing.
    pm_hit = bool(PM_PAT.search(s))
    mkt_hit = bool(MKT_PAT.search(s))
    if pm_hit and mkt_hit:
        return "Marketing"
    if pm_hit:
        return "Project Management"
    if mkt_hit:
        return "Marketing"

    # 7) Consulting (after PM/MKT to reduce collisions with "strategy/projects")
    if CONSULTING_PAT.search(s):
        return "Consulting"

    # 😍 Administrative
    if ADMIN_PAT.search(s):
        return "Administrative"

    return "Other"

# -----------------------
# 5) Evaluate with sklearn (train/test split)
# -----------------------
train_df, test_df = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df["label"]
)

test_pred = test_df["text"].map(predict_department)

print("Classes:", sorted(df["label"].unique()))
print("\nClassification report:\n")
print(classification_report(test_df["label"], test_pred, digits=2))

print("\nConfusion matrix (rows=true, cols=pred):\n")
labels_sorted = sorted(df["label"].unique())
cm = confusion_matrix(test_df["label"], test_pred, labels=labels_sorted)
cm_df = pd.DataFrame(cm, index=[f"true_{l}" for l in labels_sorted],
                        columns=[f"pred_{l}" for l in labels_sorted])
print(cm_df)

# -----------------------
# 6) (Optional) Inspect top error pairs
# -----------------------
err = test_df.assign(pred=test_pred)
err = err[err["label"] != err["pred"]]
if len(err):
    top_pair = err.groupby(["label", "pred"]).size().sort_values(ascending=False).index[0]
    true_lab, pred_lab = top_pair
    print(f"\nTop error pair: true='{true_lab}' -> pred='{pred_lab}'")
    print("Examples:")
    print(err[(err["label"] == true_lab) & (err["pred"] == pred_lab)]["text"].head(25).to_list())
else:
    print("\nNo errors in this split (unlikely, but possible on small data).")

Classes: ['Administrative', 'Business Development', 'Consulting', 'Customer Support', 'Human Resources', 'Information Technology', 'Marketing', 'Other', 'Project Management', 'Purchasing', 'Sales']

Classification report:

                        precision    recall  f1-score   support

        Administrative       0.57      1.00      0.72        17
  Business Development       0.58      0.48      0.53       124
            Consulting       0.83      0.61      0.70        33
      Customer Support       0.11      0.43      0.17         7
       Human Resources       0.38      1.00      0.55         6
Information Technology       0.59      0.51      0.55       261
             Marketing       0.85      0.68      0.76       859
                 Other       0.03      1.00      0.06         8
    Project Management       0.50      0.28      0.35        40
            Purchasing       0.70      0.88      0.78         8
                 Sales       0.76      0.73      0.74       666

       

In [37]:
import re, unicodedata

def normalize_title(title: str) -> str:
    s = str(title).lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = re.sub(r"[^\w\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

DEPARTMENT_RULES = [
    ("Information Technology", re.compile(
        r"\b(it|information technology|software|developer|engineer|data|cloud|devops|ml|ai|analytics)\b"
    )),
    ("Marketing", re.compile(
        r"\b(marketing|brand|growth|seo|content|performance|kommunikation)\b"
    )),
    ("Sales", re.compile(
        r"\b(sales|vertrieb|account|business development|bd)\b"
    )),
    ("Finance", re.compile(
        r"\b(finance|financial|accounting|controlling|audit|treasury)\b"
    )),
    ("Human Resources", re.compile(
        r"\b(hr|human resources|recruiting|talent|people)\b"
    )),
    ("Product", re.compile(
        r"\b(product manager|product owner|product management)\b"
    )),
    ("Operations", re.compile(
        r"\b(operations|supply chain|logistics|procurement|process)\b"
    )),
    ("Customer Support", re.compile(
        r"\b(support|customer success|customer service|service desk)\b"
    )),
    ("Legal", re.compile(
        r"\b(legal|compliance|jurist|law)\b"
    )),
]

def predict_department(position: str) -> str:
    s = normalize_title(position)
    for label, pat in DEPARTMENT_RULES:
        if pat.search(s):
            return label
    return "Other"

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(
    df["text"], df["label"],
    test_size=0.2, random_state=42, stratify=df["label"]
)

pred = X_test.map(predict_department)
print(classification_report(y_test, pred))

                        precision    recall  f1-score   support

        Administrative       0.00      0.00      0.00        17
  Business Development       0.00      0.00      0.00       124
            Consulting       0.00      0.00      0.00        33
      Customer Support       0.40      0.29      0.33         7
               Finance       0.00      0.00      0.00         0
       Human Resources       0.75      1.00      0.86         6
Information Technology       0.60      0.42      0.50       261
             Marketing       1.00      0.86      0.92       859
            Operations       0.00      0.00      0.00         0
                 Other       0.00      0.00      0.00         8
               Product       0.00      0.00      0.00         0
    Project Management       0.00      0.00      0.00        40
            Purchasing       0.00      0.00      0.00         8
                 Sales       0.89      0.78      0.83       666

              accuracy                

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_

In [39]:
df["label"].value_counts()

,count
label,
Marketing,4295
Sales,3328
Information Technology,1305
Business Development,620
Project Management,201
Consulting,167
Administrative,83
Other,42
Purchasing,40


In [40]:
import re, unicodedata

def normalize_title(title: str) -> str:
    s = str(title).lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = re.sub(r"[^\w\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

RULES = [
    # Information Technology
    ("Information Technology", re.compile(
        r"\b(it|information technology|software|developer|engineer|data|analytics|cloud|devops|ml|ai)\b"
    )),

    # Marketing
    ("Marketing", re.compile(
        r"\b(marketing|brand|growth|seo|content|performance|kommunikation|crm)\b"
    )),

    # Sales
    ("Sales", re.compile(
        r"\b(sales|vertrieb|account manager|key account|sales manager|sales representative)\b"
    )),

    # Business Development
    ("Business Development", re.compile(
        r"\b(business development|bd manager|partnerships|alliances|strategic growth)\b"
    )),

    # Project Management
    ("Project Management", re.compile(
        r"\b(project manager|projektleiter|program manager|pmo)\b"
    )),

    # Consulting
    ("Consulting", re.compile(
        r"\b(consultant|consulting|advisory|advisor)\b"
    )),

    # Human Resources
    ("Human Resources", re.compile(
        r"\b(hr|human resources|recruiting|talent|people)\b"
    )),

    # Purchasing
    ("Purchasing", re.compile(
        r"\b(purchasing|procurement|einkauf|sourcing)\b"
    )),

    # Customer Support
    ("Customer Support", re.compile(
        r"\b(customer support|customer service|support|helpdesk|service desk|customer success)\b"
    )),

    # Administrative
    ("Administrative", re.compile(
        r"\b(assistant|administration|administrative|office management|back office)\b"
    )),
]

def predict_department(position: str) -> str:
    s = normalize_title(position)
    for label, pat in RULES:
        if pat.search(s):
            return label
    return "Other"

In [41]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(
    df["text"], df["label"],
    test_size=0.2, random_state=42, stratify=df["label"]
)

pred = X_test.map(predict_department)
print(classification_report(y_test, pred))

                        precision    recall  f1-score   support

        Administrative       0.22      0.12      0.15        17
  Business Development       0.96      0.41      0.58       124
            Consulting       0.79      0.67      0.72        33
      Customer Support       0.50      0.29      0.36         7
       Human Resources       0.86      1.00      0.92         6
Information Technology       0.60      0.42      0.50       261
             Marketing       0.85      0.86      0.85       859
                 Other       0.02      1.00      0.05         8
    Project Management       0.42      0.20      0.27        40
            Purchasing       1.00      0.88      0.93         8
                 Sales       0.98      0.76      0.86       666

              accuracy                           0.72      2029
             macro avg       0.65      0.60      0.56      2029
          weighted avg       0.85      0.72      0.77      2029

